## 目的：population annealingを全解探索で確認

- データ
```python
'training_markers.txt','training_yields.txt'
```

- 予測アルゴリズム

χ2分類    "説明変数がカテゴリーで目的変数もカテゴリーのときは、χ2乗検定の独立性検定を用いる。"

- 最高温度

2500

母集団から標本を抜き取ることを標本抽出（またはサンプリング）といいます。標本抽出には、復元抽出と非復元抽出があります。

「復元抽出」とは

- なんどかの抽出をする際、抽出したものを1個ずつもとの集団に戻してから、次の抽出を行っていく方法です。
- 母集団の1個1個の要素は互いに独立しています。たとえば、1から10までの番号が書かれたボールが壺の中に入っており、ボール1個を取りだすとします。このとき、1から10番目まで、どのボールも取りだされる確率は同じ、1/10です。5番のボールを取りだしたとして、これをもとに戻します。次に、ふたたびボールを抜き取るときに、5番のボールを引き当てる確率もその他の番号を抜き取る確率も、変わらず1/10なままです。各要素間での独立性が保たれています。1回目の抜き取りは、2回目の抜き取りに影響を与えません。



In [1]:
#import
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import random, math, warnings
from sklearn.feature_selection import SelectKBest, chi2
import pprint
import itertools

In [2]:
#valiable
num_select = 3

MCS = 1

#降順である{1/b}の作成
Max_temper = 2500 #最高温度（スタート地点）
b = [1/j for j in reversed(range(1, Max_temper+1))]
print("1/bのトップバッター(最大値希望)1/b[0]は{}である".format(1/b[0]))
print("1/bのラスト1/b[-1]（最小値希望）は{}である".format(1/b[-1]))

1/bのトップバッター(最大値希望)1/b[0]は2500.0である
1/bのラスト1/b[-1]（最小値希望）は1.0である


In [3]:
X = pd.read_table('training_markers.txt', header=None,  sep ='\t')# 説明変数（こっちの特徴量上位5つ取り出す）
print('the following is a part of X')
#pprint.pprint(X.head())
print("X.shape is {}".format(X.shape))
print('type of X element is {}'.format(type(X[0][0])))

data_yields = pd.read_table('training_yields.txt', header=None,  sep ='\t')# 目的変数
ori_y = data_yields.T.iloc[0]
print('type of data_yields is {}'.format(type(data_yields)))
print('type of ori_y is {}'.format(type(ori_y)))

num_sample = X.shape[0]#サンプル数
num_feature = X.shape[1]#特徴量の数
K = 2**num_sample

the following is a part of X
X.shape is (20, 20)
type of X element is <class 'numpy.int64'>
type of data_yields is <class 'pandas.core.frame.DataFrame'>
type of ori_y is <class 'pandas.core.series.Series'>


In [4]:
#状態y(yの色んなパターン)をK個作る
y_element = range(2) #0, 1なので
y_tuple_list = list(itertools.product(y_element, repeat=num_sample))

print("type of y_tuple_list's element is {}".format(type(y_tuple_list[0])))
print(type(pd.Series(y_tuple_list[0]))) #ori_yとtypeを揃えるためこのようにしてseriesにしないといけない
len(y_tuple_list)==K

type of y_tuple_list's element is <class 'tuple'>
<class 'pandas.core.series.Series'>


True

In [5]:
#mask_oriの計算（以降のエネルギー計算の基準となる）
#ori_y == yとなる場合だけ、E(y)==0となる仕様

selector = SelectKBest(score_func=chi2, k=num_select) #SelectKBest #chi2...分類
selector.fit(X, ori_y)
mask_ori = selector.get_support() #各特徴量を選択したか否かのmaskを取得

In [10]:
ori_y

0     1
1     0
2     0
3     1
4     1
5     0
6     0
7     1
8     0
9     1
10    0
11    0
12    1
13    0
14    0
15    0
16    1
17    1
18    1
19    1
Name: 0, dtype: int64

In [12]:
#エネルギーが0(最低)となるのがいつか基準を先に計算しておきましょう

def E(y): #毎回のmaskとyが引数、合致度合いの計算を行う
    selector = SelectKBest(score_func=chi2, k=num_select) #SelectKBest #chi2...分類
    selector.fit(X, y)
    mask_each = selector.get_support() #各特徴量を選択したか否かのmaskを取得
    
    energy_trait_sum = 0
    for i in range(len(mask_each)):
        if mask_each[i]!=mask_ori[i]:
            energy_trait_sum += 1
    return energy_trait_sum #もとのyのときのmask(これをmaskとしている)との合致度合い（大きい⇒よく合致している）

In [13]:
E(ori_y)

0

In [15]:
Ene0_list = []
for k in range(K):
    if E(pd.Series(y_tuple_list[k])) == 0:
        Ene0_list.append(k)

print("エネルギーが0のyは{}個ある".format(len(Ene0_list)))

エネルギーが0のyは2870個ある
